In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

In [4]:
df_paths = [
    '../input/ico-coffee-dataset-worldwide/domestic-consumption.csv',
    '../input/ico-coffee-dataset-worldwide/exports-calendar-year.csv',
    '../input/ico-coffee-dataset-worldwide/exports-crop-year.csv',
    '../input/ico-coffee-dataset-worldwide/gross-opening-stocks.csv',
    '../input/ico-coffee-dataset-worldwide/total-production.csv'
]

In [8]:
dfs = [pd.read_csv(df_path) for df_path in df_paths]

In [15]:
def get_means(df):
    df = df.copy()
    countries = df[df.columns[0]]
#     print(countries)
    means = df.mean(axis = 1)
    df = pd.concat([countries, means], axis = 1)
    df.columns = ['country', countries.name]
    return df

In [21]:
# get_means(dfs[0])

In [19]:
def make_df(dfs):
    processed_dfs = []
    for df in dfs:
        processed_dfs.append(get_means(df))
        
    df = processed_dfs[0]
    
    for i in range(1, len(processed_dfs)):
        df = df.merge(processed_dfs[i], on = 'country')
    
    return df

In [20]:
data = make_df(dfs)
data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """


,country,domestic_consumption,exports,exports_crop_year,gross_opening_stocks,total_production
0,Angola,25.689655,24.115531,23.187083,42.068966,45.256048
1,Bolivia (Plurinational State of),41.103448,78.384152,76.904748,7.237931,116.146128
2,Brazil,15234.310345,25706.195606,25919.128803,23213.206897,41067.783976
3,Burundi,1.891966,363.186423,356.056321,48.979690,350.172424
4,Ecuador,214.137931,1115.800914,1105.434110,108.592069,1076.504352
5,Indonesia,2662.137931,5878.047357,5879.061059,690.114655,8452.302438
6,Madagascar,325.405724,289.048949,280.822603,87.587000,587.719424
7,Malawi,1.310345,47.247465,46.095669,4.896552,46.716359
8,Papua New Guinea,2.004828,1015.455512,1015.019583,55.757928,1010.334755
9,Paraguay,19.482759,25.757542,20.850776,36.379310,34.678362


In [23]:
# get_means(dfs[0]).merge(get_means(dfs[1]))

In [22]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop country column
    df = df.drop('country', axis=1)
    
    # Split df into X and y
    y = df['total_production']
    X = df.drop('total_production', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [24]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [25]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
print("Model trained.")

Model trained.


In [26]:
y_pred = model.predict(X_test)

rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print("RMSE: {:.2f}".format(rmse))

r2 = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
print("R^2: {:.5f}".format(r2))

RMSE: 7637.68
R^2: 0.37201
